# The Price of Hedging
1. Below we first generate 2 random return series we will use in the homwork.

In [53]:
import numpy as np 
import pandas as pd 
def gen_strat_returns():
    np.random.seed(5)
    

    corr = [[1, -0.5],
            [-0.5, 1]]
    
    corr = np.array(corr)
    
    vols = np.diag(np.array([0.1, 0.1])) / np.sqrt(252)
    
    sigma = vols @ corr @ vols
    
    mu = np.array([0.1,-0.05]) / 252
    
    dates = pd.date_range('20100101','20191231',freq='B')
    
    rets = np.random.multivariate_normal(mu, sigma, size = len(dates))
    rets = pd.DataFrame(rets,columns = ['X','HEDGE'], index = dates)
    

    rets = rets/rets.std()*0.1/np.sqrt(252)
    rets = rets-rets.mean()
    rets = rets+mu
    return rets

rets = gen_strat_returns()

2. Compute some basic stats on the return streams, including annualized returns, volatilities, sharpes and correlations.


In [54]:
summary = {}
summary['RETURN'] = rets.mean()*252
summary['VOL'] = rets.std()*np.sqrt(252)
summary['SHARPE'] = summary['RETURN']/summary['VOL']
summary['CORR'] = rets.corr().iloc[0,1]
summary = pd.DataFrame(summary)
summary

,RETURN,VOL,SHARPE,CORR
X,0.10,0.1,1.0,-0.513782
HEDGE,-0.05,0.1,-0.5,-0.513782


3. Combine the two returns streams optimally. What is the Sharpe ratio of the combo?


In [55]:
def get_opt_weighting(rets):
    sigma = rets.cov()
    mu = rets.mean()
    wgt = np.linalg.inv(sigma).dot(mu)
    wgt = wgt/np.abs(wgt).sum()
    return wgt

wgt = get_opt_weighting(rets)
wgt

array([0.98179078, 0.01820922])

In [58]:
new_ret = rets.dot(wgt)
new_summary = {}
new_summary['RETURN'] = new_ret.mean()*252
new_summary['VOL'] = new_ret.std()*np.sqrt(252)
new_summary['SHARPE'] = new_summary['RETURN']/new_summary['VOL']
new_summary = pd.Series(new_summary)
new_summary

RETURN    0.097269
VOL       0.097256
SHARPE    1.000129
dtype: float64

4. Now, compute what happens to the optimal weights and sharpes as we change the average annualized return on "HEDGE" in a range between -10% and 10% in increments of 1%. Roughly what return do you need on "HEDGE" to justify a zero weight? 

Answer: -5%